In [2]:
import json
import numpy as np
import pandas as pd
import os
import glob
import shutil

In [3]:
json_open = open('../json_data/response_1660607286931.json', 'r')
json_load = json.load(json_open)

In [18]:
json_load

[{'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['結節'],
   'mention': '結節',
   'start_idx': 10,
   'end_idx': 10,
   'sui': 'OBJ004402',
   'certainty_scale': 4},
  'modifiers': [{'name': 'Anatomical_entity',
    'tokens': ['右', '肺', '野'],
    'mention': '右肺野',
    'start_idx': 0,
    'end_idx': 2,
    'sui': None,
    'certainty_scale': None},
   {'name': 'Size_descriptor',
    'tokens': ['３', 'cm', '大'],
    'mention': '３cm大',
    'start_idx': 4,
    'end_idx': 6,
    'sui': None,
    'certainty_scale': None},
   {'name': 'Characteristics_descriptor',
    'tokens': ['不整', 'な'],
    'mention': '不整な',
    'start_idx': 8,
    'end_idx': 9,
    'sui': None,
    'certainty_scale': None},
   {'name': 'Change_descriptor',
    'tokens': ['前回', 'より', '増大', 'し', 'て', 'いる'],
    'mention': '前回より増大している',
    'start_idx': 15,
    'end_idx': 20,
    'sui': 'CHG018603',
    'certainty_scale': None},
   {'name': 'Clinical_finding',
    'tokens': ['肺癌'],
    'mention': '肺癌',
    'st

In [19]:
json_open = open('../json_data/response_1660614630854.json', 'r')
json_load = json.load(json_open)

In [20]:
json_load

[{'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['小', '嚢胞'],
   'mention': '小嚢胞',
   'start_idx': 7,
   'end_idx': 8,
   'sui': 'OBJ002760',
   'certainty_scale': 4},
  'modifiers': [{'name': 'Anatomical_entity',
    'tokens': ['肝'],
    'mention': '肝',
    'start_idx': 0,
    'end_idx': 0,
    'sui': None,
    'certainty_scale': None},
   {'name': 'Anatomical_entity',
    'tokens': ['S', '７', '／', '８'],
    'mention': 'S７／８',
    'start_idx': 2,
    'end_idx': 5,
    'sui': None,
    'certainty_scale': None}]},
 {'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['胆石'],
   'mention': '胆石',
   'start_idx': 20,
   'end_idx': 20,
   'sui': 'OBJ004923',
   'certainty_scale': 4},
  'modifiers': [{'name': 'Anatomical_entity',
    'tokens': ['胆嚢'],
    'mention': '胆嚢',
    'start_idx': 18,
    'end_idx': 18,
    'sui': None,
    'certainty_scale': None}]},
 {'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['嚢胞'],
   'mention': '嚢胞',
   'start_i